In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

In [3]:
# Create engine: engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
# Execute query and store records in DataFrame: df
df = pd.read_sql_query("SELECT * FROM Facilities", engine)
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [5]:
query = """SELECT F.name, SUM(CASE 
                                WHEN B.memid=0 THEN B.slots*F.guestcost 
                                ELSE B.slots*F.membercost END) 
                                AS revenue
        FROM Facilities AS F
        INNER JOIN Bookings AS B ON F.facid = B.facid
        GROUP BY F.name
        HAVING revenue < 1000
        ORDER BY revenue DESC;"""

In [6]:
df = pd.read_sql_query(query, engine)
df.head()

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [7]:
query = '''SELECT mem.surname,mem.firstname,
    (rec.surname ||' '||rec.firstname)as recommender
    FROM Members AS mem 
    INNER JOIN Members rec
    WHERE mem.recommendedby = rec.memid AND mem.recommendedby > 0 
    ORDER BY mem.surname,mem.firstname;'''

In [8]:
df = pd.read_sql_query(query, engine)
df.head()

,surname,firstname,recommender
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren


**Q12: Find the facilities with their usage by member, but not guests**

In [9]:
query = '''SELECT F.name AS name, SUM(B.slots) AS usage 
    FROM Bookings AS B 
    LEFT JOIN Facilities AS F ON F.facid = B.facid 
    WHERE B.memid > 0 
    GROUP BY name 
    ORDER BY usage DESC'''

In [10]:
df = pd.read_sql_query(query, engine)
df.head()

,name,usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860


**Q13: Find the facilities usage by month, but not guests**

In [11]:
query = '''SELECT F.name AS name, 
      SUM(B.slots) AS usage,
      strftime('%m',date(starttime))as month
      FROM 
      Bookings AS B 
      LEFT JOIN Facilities AS F ON F.facid = B.facid 
      WHERE B.memid > 0 
      GROUP BY name,month 
      ORDER BY usage DESC;'''

In [12]:
df = pd.read_sql_query(query, engine)
df.head()

,name,usage,month
0,Badminton Court,507,09
1,Pool Table,443,09
2,Tennis Court 1,417,09
3,Badminton Court,414,08
4,Tennis Court 2,414,09
